# Assignment on Capstone project related to "Neighborhoods in Toronto!"

## To build the code to scrape the following Wikipedia page to transform data into dataframe. 
## Wikipedia page link-https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M) 

In [1]:
# Impoerting all necessary libraries 
from bs4 import BeautifulSoup
!pip install beautifulsoup4
import requests
import lxml
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis

        

In [2]:
# Reading Source
htmlget = requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M').text

soup = BeautifulSoup(htmlget, 'lxml')

type(soup)

bs4.BeautifulSoup

In [3]:
# segregate the table tag to build header and detail records 
table  = soup.find('table')
Header=[]
type(table)


bs4.element.Tag

In [4]:
# segregate tr tag to build header 
tab=table.tr
tab


<tr>
<th>Postcode</th>
<th>Borough</th>
<th>Neighbourhood
</th></tr>

In [5]:
# Build header for Dataframe 
Header=[]
for i in tab: 
    if 'Tag' in str(type(i)): 
        Header.append(i.text.rstrip())
        
type(Header)

Header

['Postcode', 'Borough', 'Neighbourhood']

In [6]:
Detail=[]
rec=table.tbody
for i in rec: 
    if 'Tag' in str(type(i)):
        for j in i: 
            if 'Tag' in str(type(j)):
                   Detail.append(j.text.rstrip())
                    
Detail[0:10]

['Postcode',
 'Borough',
 'Neighbourhood',
 'M1A',
 'Not assigned',
 'Not assigned',
 'M2A',
 'Not assigned',
 'Not assigned',
 'M3A']

In [7]:
detail_rec= Detail[3:] # to bypass Header 
# convert into a table 

Detail_records=np.array(detail_rec).reshape(int(len(detail_rec)/3),3)


Toronto_data=pd.DataFrame(columns=Header, data=Detail_records)
Toronto_data.head()


,Postcode,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront


In [8]:
Toronto_data.count() # total 289 rows are extracted into DF. 

Postcode         289
Borough          289
Neighbourhood    289
dtype: int64

## Only process the cells that have an assigned borough. Ignore cells with a borough that is Not assigned.

In [9]:
# filtering the rows that have "Not assigned" in Borough column. 
Toronto_filter=Toronto_data[(Toronto_data.Borough != "Not assigned")]
Toronto_filter.head(7)

,Postcode,Borough,Neighbourhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront
5,M5A,Downtown Toronto,Regent Park
6,M6A,North York,Lawrence Heights
7,M6A,North York,Lawrence Manor
8,M7A,Queen's Park,Not assigned


## If a cell has a borough but a Not assigned neighborhood, then the neighborhood will be the same as the borough

In [10]:
Toronto_filter.Neighbourhood.replace("Not assigned", Toronto_filter.Borough, inplace=True)
Toronto_filter.head(7)

/home/jupyterlab/conda/lib/python3.6/site-packages/pandas/core/generic.py:5890: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._update_inplace(new_data)


,Postcode,Borough,Neighbourhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront
5,M5A,Downtown Toronto,Regent Park
6,M6A,North York,Lawrence Heights
7,M6A,North York,Lawrence Manor
8,M7A,Queen's Park,Queen's Park


## More than one neighborhood can exist in one postal code area. For example, in the table on the Wikipedia page, you will notice that M5A is listed twice and has two neighborhoods: Harbourfront and Regent Park. These two rows will be combined into one row with the neighborhoods separated with a comma as shown in row 11 in the above table.

In [11]:
Toronto_filter.sort_values(['Postcode'], ascending=[True])
Toronto_fil=Toronto_filter.groupby("Postcode")["Neighbourhood"].apply(','.join).reset_index()

Toronto_final=pd.merge(Toronto_fil, Toronto_filter, on=["Postcode"])
Toronto_final.head()

,Postcode,Neighbourhood_x,Borough,Neighbourhood_y
0,M1B,"Rouge,Malvern",Scarborough,Rouge
1,M1B,"Rouge,Malvern",Scarborough,Malvern
2,M1C,"Highland Creek,Rouge Hill,Port Union",Scarborough,Highland Creek
3,M1C,"Highland Creek,Rouge Hill,Port Union",Scarborough,Rouge Hill
4,M1C,"Highland Creek,Rouge Hill,Port Union",Scarborough,Port Union


In [12]:
# Remove 4 th column and reshape the column structure
Toronto_Postcode=Toronto_final.drop(['Neighbourhood_y'], axis=1)
Toronto_Postcode.rename(columns = {'Neighbourhood_x':'Neighbourhood','Postcode' : 'PostalCode'}, inplace = True)
Toronto_Postcode.head()


,PostalCode,Neighbourhood,Borough
0,M1B,"Rouge,Malvern",Scarborough
1,M1B,"Rouge,Malvern",Scarborough
2,M1C,"Highland Creek,Rouge Hill,Port Union",Scarborough
3,M1C,"Highland Creek,Rouge Hill,Port Union",Scarborough
4,M1C,"Highland Creek,Rouge Hill,Port Union",Scarborough


In [13]:
cols = ['PostalCode', 'Borough','Neighbourhood' ]
Toronto_Postcode2=Toronto_Postcode.reindex(columns=cols)

Toronto_Final=Toronto_Postcode2.drop_duplicates(subset='PostalCode' )

Toronto_Final.head()


,PostalCode,Borough,Neighbourhood
0,M1B,Scarborough,"Rouge,Malvern"
2,M1C,Scarborough,"Highland Creek,Rouge Hill,Port Union"
5,M1E,Scarborough,"Guildwood,Morningside,West Hill"
8,M1G,Scarborough,Woburn
9,M1H,Scarborough,Cedarbrae



## In the last cell of your notebook, use the .shape method to print the number of rows of your dataframe


In [14]:
Toronto_Final.shape

(103, 3)

## Use the Geocoder package or the csv file to create the following dataframe:

In [15]:
# Create a DataFrame and read CSV as Geocoder package has issues
postalcode_Geo = pd.DataFrame(columns=['PostalCode', 'Latitude', 'Longitude'])
path='http://cocl.us/Geospatial_data' # Path location 

postalcode_Geo=pd.read_csv(path)

postalcode_Geo.count()


Postal Code    103
Latitude       103
Longitude      103
dtype: int64

In [16]:
postalcode_Geo.rename(columns = {'Postal Code' : 'PostalCode'}, inplace = True)
postalcode_Geo.head()

,PostalCode,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [17]:
# Merging two DF's on PostalCode after renaming columns to match. 

Toronto_Geo_Final=pd.merge(Toronto_Final, postalcode_Geo, on=["PostalCode"])
Toronto_Geo_Final.head()

,PostalCode,Borough,Neighbourhood,Latitude,Longitude
0,M1B,Scarborough,"Rouge,Malvern",43.806686,-79.194353
1,M1C,Scarborough,"Highland Creek,Rouge Hill,Port Union",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood,Morningside,West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476


## To generate maps to visualize your neighborhoods and how they cluster together

In [18]:
# Importing Necessary libraries for generating map
import json # library to handle JSON files

!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

print('Libraries imported.')


Solving environment: done

## Package Plan ##

  environment location: /home/jupyterlab/conda

  added / updated specs: 
    - geopy


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    geopy-1.18.1               |             py_0          51 KB  conda-forge
    openssl-1.0.2p             |       h470a237_2         3.1 MB  conda-forge
    geographiclib-1.49         |             py_0          32 KB  conda-forge
    ------------------------------------------------------------
                                           Total:         3.2 MB

The following NEW packages will be INSTALLED:

    geographiclib: 1.49-py_0         conda-forge
    geopy:         1.18.1-py_0       conda-forge

The following packages will be UPDATED:

    openssl:       1.0.2p-h470a237_1 conda-forge --> 1.0.2p-h470a237_2 conda-forge


geopy-1.18.1         | 51 KB     | ##################################### | 100% 
ope

In [20]:
print('The dataframe has {} boroughs and {} neighborhoods.'.format(
        len(Toronto_Geo_Final['Borough'].unique()),
        Toronto_Geo_Final.shape[0]
    )
)

The dataframe has 11 boroughs and 103 neighborhoods.


In [21]:
Toronto_Geo_Final['Borough'].unique()

array(['Scarborough', 'North York', 'East York', 'East Toronto',
       'Central Toronto', 'Downtown Toronto', 'York', 'West Toronto',
       "Queen's Park", 'Mississauga', 'Etobicoke'], dtype=object)

In [22]:
# filtering Borough which contains "Toronto"

Toronto_Only= Toronto_Geo_Final[Toronto_Geo_Final['Borough'].str.contains('Toronto')]

In [25]:
Toronto_Only.head()

,PostalCode,Borough,Neighbourhood,Latitude,Longitude
37,M4E,East Toronto,The Beaches,43.676357,-79.293031
41,M4K,East Toronto,"The Danforth West,Riverdale",43.679557,-79.352188
42,M4L,East Toronto,"The Beaches West,India Bazaar",43.668999,-79.315572
43,M4M,East Toronto,Studio District,43.659526,-79.340923
44,M4N,Central Toronto,Lawrence Park,43.728020,-79.388790


In [27]:
# Total 38 Postal codes fetched for Borough Toronto 
# Fetching Latitude and Longitude for Toronto.
address = 'Toronto, ON'

geolocator = Nominatim()
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto City are {}, {}.'.format(latitude, longitude))


/home/jupyterlab/conda/lib/python3.6/site-packages/ipykernel_launcher.py:5: DeprecationWarning: Using Nominatim with the default "geopy/1.18.1" `user_agent` is strongly discouraged, as it violates Nominatim's ToS https://operations.osmfoundation.org/policies/nominatim/ and may possibly cause 403 and 429 HTTP errors. Please specify a custom `user_agent` with `Nominatim(user_agent="my-application")` or by overriding the default `user_agent`: `geopy.geocoders.options.default_user_agent = "my-application"`. In geopy 2.0 this will become an exception.
  """


The geograpical coordinate of Toronto City are 43.653963, -79.387207.


# Create a map of Toronto with neighborhoods superimposed on top.

In [29]:
# create map of New York using latitude and longitude values
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(Toronto_Only['Latitude'], Toronto_Only['Longitude'], Toronto_Only['Borough'], Toronto_Only['Neighbourhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto